In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import uproot 
import awkward as ak
from pathlib import Path

from typing import Dict, List 
import re
import pickle
from tqdm import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split



In [2]:
training_vars = ['jet_pt', 'jet_nTracks', 'jet_trackWidth', 'jet_trackC1']
training_weight = ['equal_weight']
gbdt_filename = './gbdt.model'


# Load Data

In [3]:
sample_path = '../../../samples/sample_allpt_12M_jets.pkl'
sample = pd.read_pickle(sample_path)

In [4]:
sample.columns

Index(['jet_pt', 'jet_eta', 'jet_nTracks', 'jet_trackWidth', 'jet_trackC1',
       'jet_trackBDT', 'jet_PartonTruthLabelID', 'equal_weight',
       'event_weight', 'flatpt_weight', 'is_forward', 'target', 'pt_idx'],
      dtype='object')

# Train with default config

In [5]:
X = sample.iloc[:, :-1]
y = sample.iloc[:, -1]

X_dev,X_eval, y_dev,y_eval = train_test_split(X, y, test_size=0.1, random_state=456)
X_train, X_test, y_train, y_test = train_test_split(X_dev, y_dev, test_size=0.1/0.9, random_state=789)

In [6]:
event_weight_idx = X.columns.get_loc('event_weight')
equal_weight_idx = X.columns.get_loc('equal_weight')
flatpt_weight_idx = X.columns.get_loc('flatpt_weight')

In [7]:
X_train[training_vars].head()

,jet_pt,jet_nTracks,jet_trackWidth,jet_trackC1
8091347,531.234863,13.0,0.014092,0.166602
6210831,1103.663452,25.0,0.031219,0.216386
1524473,1442.918823,15.0,0.014858,0.088302
13948873,523.586853,15.0,0.062496,0.134028
12416991,540.538818,16.0,0.053047,0.196087


In [8]:
X_train[training_weight].head()

,equal_weight
8091347,1.0
6210831,1.0
1524473,1.0
13948873,1.0
12416991,1.0


In [9]:
bdt = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=42, verbose=1)

bdt.fit(X_train[training_vars], y_train, sample_weight=X_train[training_weight].to_numpy().flatten())

### 

      Iter       Train Loss   Remaining Time 
         1           0.0123          124.30m
         2           0.0023          126.96m
         3           0.0004          128.29m
         4           0.0001          126.71m
         5           0.0000          121.04m
         6           0.0000          114.57m


In [ ]:
pickle.dump(bdt, open(gbdt_filename, 'wb'))

# Predict on test datasets

In [ ]:
y_test_decisions = bdt.decision_function(X_test[training_vars])
y_test_gluon_id = np.where(y_test==1)[0]
y_test_quark_id = np.where(y_test==0)[0]
bins_scores = np.linspace(-1, 1, 101)
plt.hist(y_test_decisions[y_test_gluon_id], weights=X_test.iloc[y_test_gluon_id, X_test.columns.get_loc('event_weight')], bins= bins_scores, alpha=0.5, label='gluon'+f"_num: {len(y_test_gluon_id)}", color = 'blue') # add the weights! 
plt.hist(y_test_decisions[y_test_quark_id], weights=X_test.iloc[y_test_quark_id, X_test.columns.get_loc('event_weight')], bins= bins_scores, alpha=0.5, label='quark'+f"_num: {len(y_test_quark_id)}", color = 'red')
plt.legend(loc='upper left')
plt.text(0.05, 0.75, f"num: {len(X_test)}", transform=plt.gca().transAxes)
plt.xlabel("BDT Decision Function")
plt.ylabel("number of jets")
plt.title(r"New Training with event weight")
plt.show()

# ROC

In [ ]:
from sklearn.metrics import roc_curve, auc

def Draw_ROC_all(y, decisions, y_tmva, y_ntrk, X_weight, features):
    # Compute ROC curve and area under the curve
    fpr, tpr, thresholds = roc_curve(y, decisions, sample_weight = X_weight)
    fpr_tmva, tpr_tmva, thresholds_tmva = roc_curve(y, y_tmva, sample_weight = X_weight)
    fpr_ntrk, tpr_ntrk, thresholds_ntrk =  roc_curve(y, y_ntrk, sample_weight = X_weight)

    roc_auc = auc(fpr, tpr)
    roc_auc_tmva = auc(fpr_tmva, tpr_tmva)
    roc_auc_ntrk = auc(fpr_ntrk, tpr_ntrk)


    plt.plot(1-fpr, tpr, lw=1, label='ROC_NewTraining (area = %0.3f)'%(roc_auc))
    plt.plot(1-fpr_tmva, tpr_tmva, lw=1, label='ROC_TMVA (area = %0.3f)'%(roc_auc_tmva))
    plt.plot(1-fpr_ntrk, tpr_ntrk, lw=1, label='ROC_Ntrk (area = %0.3f)'%(roc_auc_ntrk))

    plt.plot([0, 1], [1, 0], '--', color=(0.6, 0.6, 0.6), label='Luck')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC with features:{features}'+r" physical $p_{T}$ weight")
    plt.legend(loc="lower left")
    plt.grid()
    plt.savefig("ROC.png")
    plt.show()

Draw_ROC_all(y_test, y_test_decisions, y_tmva=X_test.iloc[:,X_test.columns.get_loc('jet_trackBDT')] , y_ntrk=X_test.iloc[:,X_test.columns.get_loc('jet_nTracks')], X_weight=X_test['event_weight'], features="Only Ntrk")


# Overtrain validation

In [ ]:
y_test_decisions = bdt.decision_function(X_test[training_vars])
y_train_decisions = bdt.decision_function(X_train[training_vars])


y_test_decisions_gluon = y_test_decisions[np.where(y_test==1)]
y_test_decisions_quark = y_test_decisions[np.where(y_test==0)]
y_train_decisions_gluon = y_train_decisions[np.where(y_train==1)]
y_train_decisions_quark = y_train_decisions[np.where(y_train==0)]
decisions = []
decisions.append(y_train_decisions_gluon)
decisions.append(y_train_decisions_quark)
decisions.append(y_test_decisions_gluon)
decisions.append(y_test_decisions_quark)

In [ ]:
custom_bins = np.linspace(-1, 1, 101)
custom_bin_width = (custom_bins[1] - custom_bins[0])
custom_bin_center = (custom_bins[:-1] + custom_bins[1:]) / 2

fig, ax = plt.subplots()
ax.hist(decisions[0], weights = X_train.iloc[np.where(y_train==1)[0], event_weight_idx],
            color='b', alpha=0.5, bins=custom_bins,
            histtype='stepfilled', density=True,
            label='Gluon (train)')
ax.hist(decisions[1], weights = X_train.iloc[np.where(y_train==0)[0], event_weight_idx],
            color='r', alpha=0.5, bins=custom_bins,
            histtype='stepfilled', density=True,
            label='Quark (train)')

hist, bins = np.histogram(decisions[2], weights=X_test.iloc[np.where(y_test==1)[0], event_weight_idx],
                            bins=custom_bins,  density=True)
scale = len(decisions[2]) / sum(hist)
err = np.sqrt(hist * scale) / scale

plt.errorbar(custom_bin_center, hist, yerr=err, fmt='o', c='b', label='Gluon (test)')

hist, bins = np.histogram(decisions[3], weights=X_test.iloc[np.where(y_test==0)[0], event_weight_idx],
                            bins=custom_bins, density=True)
scale = len(decisions[2]) / sum(hist)
err = np.sqrt(hist * scale) / scale

plt.errorbar(custom_bin_center, hist, yerr=err, fmt='o', c='r', label='Quark (test)')
plt.xlabel("BDT output")
plt.ylabel("Arbitrary units")
plt.legend(loc='best')
plt.title("Overtraining test")

In [ ]:
y_sample = bdt.decision_function(sample[training_vars])
sample['new_BDTscore'] = y_sample

features = [*sample.columns[:6]] + ['new_BDTscore'] 
HistBins = {
    features[0] : np.linspace(0, 2000, 61), 
    features[1] : np.linspace(-2.5, 2.5, 51),
    features[2] : np.linspace(0, 60, 61),
    features[3] : np.linspace(0, 0.4, 61), 
    features[4] : np.linspace(0, 0.4, 61), 
    features[5] : np.linspace(-1.0, 1.0, 51), 
    features[6] : np.linspace(-1.0, 1.0, 51),
}

sample_forward = sample[sample['is_forward']==1]
sample_central = sample[sample['is_forward']==0]

for feature_name in features:
    custom_bins = HistBins[feature_name]
    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize = (10,5))
    ax0.hist(sample_forward[feature_name], bins = custom_bins, weights = sample_forward['event_weight'], label = f'forward, num: {len(sample_forward)}', color='red', alpha = 0.5)
    ax0.hist(sample_central[feature_name], bins = custom_bins, weights = sample_central['event_weight'], label = f'gluon, num: {len(sample_central)}', color='blue', alpha = 0.5)
    ax0.legend()
    ax0.set_xlabel(feature_name)
    ax0.set_ylabel(f"Weighted Yield")
    ax0.set_title(f"Weighted Yield Comparison")

    if feature_name == 'jet_pt':
        ax0.set_yscale('log')

    ax1.hist(sample_forward[feature_name], bins = custom_bins, weights = sample_forward['event_weight'], label = 'forward', color='red', alpha = 0.5, density=True)
    ax1.hist(sample_central[feature_name], bins = custom_bins, weights = sample_central['event_weight'], label = 'gluon', color='blue', alpha = 0.5, density=True)
    ax1.legend()
    ax1.set_xlabel(feature_name)
    ax1.set_ylabel(f"Weighted Shape")
    ax1.set_title(f"Weighted Shape Comparison")
    if feature_name == 'jet_pt':
        ax1.set_yscale('log')
    fig.suptitle(f"Forward vs Central: {feature_name}")

    plt.plot()
    

# Backup

In [ ]:
y_test_decisions = bdt.decision_function(X_eval[training_vars])

y_test_decisions_gluon = y_test_decisions[np.where(y_eval==1)]
y_test_decisions_quark = y_test_decisions[np.where(y_eval==0)]
y_train_decisions_gluon = y_train_decisions[np.where(y_train==1)]
y_train_decisions_quark = y_train_decisions[np.where(y_train==0)]
decisions = []
decisions.append(y_train_decisions_gluon)
decisions.append(y_train_decisions_quark)
decisions.append(y_test_decisions_gluon)
decisions.append(y_test_decisions_quark)


custom_bins = np.linspace(-1, 1, 101)
custom_bin_width = (custom_bins[1] - custom_bins[0])
custom_bin_center = (custom_bins[:-1] + custom_bins[1:]) / 2

fig, ax = plt.subplots()
ax.hist(decisions[0], weights = X_train.iloc[np.where(y_train==1)[0], event_weight_idx],
            color='b', alpha=0.5, bins=custom_bins,
            histtype='stepfilled', density=True,
            label='Gluon (train)')
ax.hist(decisions[1], weights = X_train.iloc[np.where(y_train==0)[0], event_weight_idx],
            color='r', alpha=0.5, bins=custom_bins,
            histtype='stepfilled', density=True,
            label='Quark (train)')

hist, bins = np.histogram(decisions[2], weights=X_eval.iloc[np.where(y_eval==1)[0], event_weight_idx],
                            bins=custom_bins,  density=True)
scale = len(decisions[2]) / sum(hist)
err = np.sqrt(hist * scale) / scale

plt.errorbar(custom_bin_center, hist, yerr=err, fmt='o', c='b', label='Gluon (test)')

hist, bins = np.histogram(decisions[3], weights=X_eval.iloc[np.where(y_eval==0)[0], event_weight_idx],
                            bins=custom_bins, density=True)
scale = len(decisions[2]) / sum(hist)
err = np.sqrt(hist * scale) / scale

plt.errorbar(custom_bin_center, hist, yerr=err, fmt='o', c='r', label='Quark (test)')
plt.xlabel("BDT output")
plt.ylabel("Arbitrary units")
plt.legend(loc='best')
plt.title("Overtraining test")



In [ ]:
bdt_filename = './BDT_explore_1500_ntrk_equalweight.model'

with open(bdt_filename, 'rb') as f:

    bdt = pickle.load(f)
    
def plot_FvsC(sample, bdt):
    global training_vars
    y_sample = bdt.decision_function(sample[training_vars])
    sample['new_BDTscore'] = y_sample

    features = [*sample.columns[:6]] + ['new_BDTscore'] 
    HistBins = {
        features[0] : np.linspace(0, 2000, 61), 
        features[1] : np.linspace(-2.5, 2.5, 51),
        features[2] : np.linspace(0, 60, 61),
        features[3] : np.linspace(0, 0.4, 61), 
        features[4] : np.linspace(0, 0.4, 61), 
        features[5] : np.linspace(-1.0, 1.0, 51), 
        features[6] : np.linspace(-1.0, 1.0, 51),
    }

    sample_forward = sample[sample['is_forward']==1]
    sample_central = sample[sample['is_forward']==0]

    for feature_name in features:
        custom_bins = HistBins[feature_name]
        fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize = (10,5))
        ax0.hist(sample_forward[feature_name], bins = custom_bins, weights = sample_forward['event_weight'], label = f'forward, num: {len(sample_forward)}', color='red', alpha = 0.5)
        ax0.hist(sample_central[feature_name], bins = custom_bins, weights = sample_central['event_weight'], label = f'gluon, num: {len(sample_central)}', color='blue', alpha = 0.5)
        ax0.legend()
        ax0.set_xlabel(feature_name)
        ax0.set_ylabel(f"Weighted Yield")
        ax0.set_title(f"Weighted Yield Comparison")

        if feature_name == 'jet_pt':
            ax0.set_yscale('log')

        ax1.hist(sample_forward[feature_name], bins = custom_bins, weights = sample_forward['event_weight'], label = 'forward', color='red', alpha = 0.5, density=True)
        ax1.hist(sample_central[feature_name], bins = custom_bins, weights = sample_central['event_weight'], label = 'gluon', color='blue', alpha = 0.5, density=True)
        ax1.legend()
        ax1.set_xlabel(feature_name)
        ax1.set_ylabel(f"Weighted Shape")
        ax1.set_title(f"Weighted Shape Comparison")
        if feature_name == 'jet_pt':
            ax1.set_yscale('log')
        fig.suptitle(f"Forward vs Central: {feature_name}")

        plt.plot()

    return sample

sample_1500_all_jets = '/global/cfs/projectdirs/atlas/hrzhao/HEP_Repo/QG_Calibration/BDT_EB4/samples/sample_1500_all_jets.pkl'

with open(sample_1500_all_jets, 'rb') as f:
    sample_1500_all_jets = pd.read_pickle(f)
sample_1500_all_jets_w_scores =  plot_FvsC(sample=sample_1500_all_jets, bdt=bdt)

In [ ]:
bdt.feature_importances_

In [ ]:
forward_quark = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==1) &(sample_1500_all_jets_w_scores['target']==0)]
forward_gluon = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==1) &(sample_1500_all_jets_w_scores['target']==1)]
central_quark = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==0) &(sample_1500_all_jets_w_scores['target']==0)]
central_gluon = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==0) &(sample_1500_all_jets_w_scores['target']==1)]
forward = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==1)]
central = sample_1500_all_jets_w_scores[(sample_1500_all_jets_w_scores['is_forward']==0)]
quark = sample_1500_all_jets_w_scores[sample_1500_all_jets_w_scores['target']==0]
gluon = sample_1500_all_jets_w_scores[sample_1500_all_jets_w_scores['target']==1]

newbdt_bins = np.linspace(-1.0, 1.0, 101)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(forward_quark['new_BDTscore'], bins=newbdt_bins, weights=forward_quark['event_weight'], 
          label=f'forward quark, num:{len(forward_quark)}', color='red', alpha = 0.5, density=True)
ax[0].hist(central_quark['new_BDTscore'], bins=newbdt_bins, weights=central_quark['event_weight'], 
          label=f'central quark, num:{len(central_quark)}', color='blue', alpha = 0.5, density=True)
ax[0].legend()
ax[0].set_title('quark: Forward vs. Central')

ax[1].hist(forward_gluon['new_BDTscore'], bins=newbdt_bins, weights=forward_gluon['event_weight'],
          label=f'forward gluon, num:{len(forward_gluon)}', color='red', alpha = 0.5, density=True)
ax[1].hist(central_gluon['new_BDTscore'], bins=newbdt_bins, weights=central_gluon['event_weight'], 
          label=f'central gluon, num:{len(central_gluon)}', color='blue', alpha = 0.5, density=True)
ax[1].set_title('gluon: Forward vs. Central')
ax[1].legend()
fig.suptitle('New BDT')


In [ ]:
n_forward_quark = len(forward_quark)
n_central_quark = len(central_quark)
n_forward = len(forward)
n_central = len(central)

frac_forward_quark = n_forward_quark / n_forward
frac_central_quark = n_central_quark / n_central

frac_forward_gluon = 1 - frac_forward_quark
frac_central_gluon = 1 - frac_central_quark

f = np.array([[frac_forward_quark,  frac_forward_gluon], [frac_central_quark, frac_central_gluon]])
f_inv = np.linalg.inv(f)


# Do a simple extraction test
# MC closure 
feature = 'new_BDTscore'
newbdt_bins = np.linspace(-1.0, 1.0, 101)
hist_forward, _ = np.histogram(forward[feature], newbdt_bins, weights=forward['event_weight'], density = True)
hist_central, _ = np.histogram(central[feature], newbdt_bins, weights=central['event_weight'], density = True)


extract_p_Quark = f_inv[0][0] * hist_forward + f_inv[0][1]* hist_central 
extract_p_Gluon = f_inv[1][0] * hist_forward + f_inv[1][1]* hist_central 

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(quark[feature], bins=newbdt_bins, weights=quark['event_weight'], 
          label=f'truth quark', color='red', alpha = 0.5, density=True)
ax[0].stairs(values=extract_p_Quark, edges=newbdt_bins, label=f'extracted quark(MC)')
ax[0].legend()
ax[0].set_title('quark: Truth vs. Extracted MC')

ax[1].hist(gluon[feature], bins=newbdt_bins, weights=gluon['event_weight'], 
          label=f'truth quark', color='blue', alpha = 0.5, density=True)
ax[1].stairs(values=extract_p_Gluon, edges=newbdt_bins, label=f'extracted gluon(MC)')
ax[1].legend()
ax[1].set_title('gluon: Truth vs. Extracted MC')

# ax[1].hist(forward_gluon['new_BDTscore'], bins=newbdt_bins, weights=forward_gluon['event_weight'],
#           label=f'forward gluon, num:{len(forward_gluon)}', color='red', alpha = 0.5, density=True)
# ax[1].hist(central_gluon['new_BDTscore'], bins=newbdt_bins, weights=central_gluon['event_weight'], 
#           label=f'central gluon, num:{len(central_gluon)}', color='blue', alpha = 0.5, density=True)
# ax[1].set_title('gluon: Forward vs. Central')
# ax[1].legend()
fig.suptitle('New BDT')


In [ ]:
# Do a simple extraction test
# MC closure 
feature = 'jet_trackBDT'
newbdt_bins = np.linspace(-1.0, 1.0, 101)
hist_forward, _ = np.histogram(forward[feature], newbdt_bins, weights=forward['event_weight'], density = True)
hist_central, _ = np.histogram(central[feature], newbdt_bins, weights=central['event_weight'], density = True)


extract_p_Quark = f_inv[0][0] * hist_forward + f_inv[0][1]* hist_central 
extract_p_Gluon = f_inv[1][0] * hist_forward + f_inv[1][1]* hist_central 

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax[0].hist(quark[feature], bins=newbdt_bins, weights=quark['event_weight'], 
          label=f'truth quark', color='red', alpha = 0.5, density=True)
ax[0].stairs(values=extract_p_Quark, edges=newbdt_bins, label=f'extracted quark(MC)')
ax[0].legend()
ax[0].set_title('quark: Truth vs. Extracted MC')

ax[1].hist(gluon[feature], bins=newbdt_bins, weights=gluon['event_weight'], 
          label=f'truth quark', color='blue', alpha = 0.5, density=True)
ax[1].stairs(values=extract_p_Gluon, edges=newbdt_bins, label=f'extracted gluon(MC)')
ax[1].legend()
ax[1].set_title('gluon: Truth vs. Extracted MC')

fig.suptitle('Old BDT')


In [ ]:
hist_forward[0], hist_central[0] 

In [ ]:
f_inv

In [ ]:
f_inv[1][0] * hist_forward[0] + f_inv[1][1]* hist_central[0] 

In [ ]:
extract_p_Gluon

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
feature = 'jet_nTracks'
ntrk_bins = np.linspace(0, 80, 81)
hist_forward, _ = np.histogram(forward[feature], ntrk_bins, weights=forward['event_weight'], density = True)
hist_central, _ = np.histogram(central[feature], ntrk_bins, weights=central['event_weight'], density = True)

extract_p_Quark = f_inv[0][0] * hist_forward + f_inv[0][1]* hist_central 
extract_p_Gluon = f_inv[1][0] * hist_forward + f_inv[1][1]* hist_central 


ax[0].hist(quark[feature], bins=ntrk_bins, weights=quark['event_weight'], 
          label=f'truth quark', color='red', alpha = 0.5, density=True)
ax[0].stairs(values=extract_p_Quark, edges=ntrk_bins, label=f'extracted quark(MC)')
ax[0].legend()
ax[0].set_title('quark: Truth vs. Extracted MC')

ax[1].hist(gluon[feature], bins=ntrk_bins, weights=gluon['event_weight'], 
          label=f'truth quark', color='blue', alpha = 0.5, density=True)
ax[1].stairs(values=extract_p_Gluon, edges=ntrk_bins, label=f'extracted gluon(MC)')
ax[1].legend()
ax[1].set_title('gluon: Truth vs. Extracted MC')

# ax[1].hist(forward_gluon['new_BDTscore'], bins=newbdt_bins, weights=forward_gluon['event_weight'],
#           label=f'forward gluon, num:{len(forward_gluon)}', color='red', alpha = 0.5, density=True)
# ax[1].hist(central_gluon['new_BDTscore'], bins=newbdt_bins, weights=central_gluon['event_weight'], 
#           label=f'central gluon, num:{len(central_gluon)}', color='blue', alpha = 0.5, density=True)
# ax[1].set_title('gluon: Forward vs. Central')
# ax[1].legend()
fig.suptitle('New BDT')

In [ ]:
f_inv

In [ ]:
hist_forward[0]

In [ ]:
hist_central[0]

In [ ]:
f_inv[1][0] * hist_forward[0] + f_inv[1][1]* hist_central[0] 

In [ ]:
np.min(extract_p_Gluon)